<a href="https://colab.research.google.com/github/Saad-Huss41/MovieRex---Movie-Recommendation-System/blob/main/Logic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
df1=pd.read_csv('/content/tmdb_5000_credits.csv')
df2=pd.read_csv('/content/tmdb_5000_movies.csv')
# Just reading the files and importing some libraries we'll need.

FileNotFoundError: [Errno 2] No such file or directory: '/content/tmdb_5000_credits.csv'

In [ ]:
df1.columns = ['id','title','cast','crew']
df2= df2.merge(df1,on='id')
df2.head(5)
# Concatenate the datasets.

In [ ]:
C= df2['vote_average'].mean()
C

In [ ]:
m= df2['vote_count'].quantile(0.9)
m

In [ ]:
q_movies = df2.copy().loc[df2['vote_count'] >= m]
q_movies.shape
#q_movies are "qualified movies", movies with a certain number (m) of votes.

In [ ]:
def weighted_rating(x, m=m, C=C):
    v = x['vote_count']
    R = x['vote_average']
    return (v/(v+m) * R) + (m/(m+v) * C)
# Calculation based on the IMDB formula


In [ ]:
q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [ ]:
q_movies = q_movies.sort_values('score', ascending=False)
q_movies[['title', 'vote_count', 'vote_average', 'score']].head(10)
# Here is our first very basic recommender system.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
# Tfidf = Term Frequency-Inverse Document Frequency. TfidfVectorizer turns docs into matrices with numerical values showing how important a word is to the doc.

In [ ]:
tfidf = TfidfVectorizer(stop_words='english')

In [ ]:
df2['overview'] = df2['overview'].fillna('')
# We want to fill any empty descriptions with an empty string.

In [ ]:
tfidf_matrix = tfidf.fit_transform(df2['overview'])
# Here we just make the matrix

In [ ]:
tfidf_matrix.shape

In [ ]:
from sklearn.metrics.pairwise import linear_kernel

In [ ]:
cosine_sim = linear_kernel(tfidf_matrix, tfidf_matrix)
# The great thing about using linear_kernel here is that it'll perform the same thing as Cosine Similarity except it wont take as long.
# As for what it does, it calculates the dot product of the two vectors.
# Basically it multiplies the two vectors by each other which will tell us how similar each element is to every other element in the Matrix

In [ ]:
cosine_sim.shape

In [ ]:
#Construct a reverse map of indices and movie titles
indices = pd.Series(df2.index, index=df2['title']).drop_duplicates()


In [ ]:
indices[:10]

In [ ]:
# We define a function to return movies with similar scores
def get_recommendations(title, cosine_sim=cosine_sim):
    idx = indices[title]

    sim_scores = list(enumerate(cosine_sim[idx]))

    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # Sort from most similar to least

    sim_scores = sim_scores[1:11]
    # 10 closest scores
    movie_indices = [i[0] for i in sim_scores]

    return df2['title'].iloc[movie_indices]


In [ ]:
print(get_recommendations('The Great Gatsby'))

In [ ]:
# Congratulations!! You have a working recommender system based with *Content Based Filtering* and specifically centered around the plot overview.

In [ ]:
from ast import literal_eval
import ast

In [ ]:
# Important note for using other datasets: cast_y and crew_x should be changed to cast or crew if thats how the columns are named in the dataset.
# In my case, I named them this way to make this code block run since I had both cast_y, cast_x and crew_y,crew_x in my df2.
features = ['cast_y', 'crew_x', 'keywords', 'genres']
for feature in features:
  df2[feature] = df2[feature].apply(lambda x: ast.literal_eval(x) if isinstance(x, str) and x.startswith('[') and x.endswith(']') else x)
  # We want to validate the list is present before we add it to avoid any errors.

In [ ]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan
    # We are getting the director of the movie.

In [ ]:
def get_list(x):
    if isinstance(x, list):
        names = [i['name'] for i in x]
        #Check if more than 3 elements exist. If yes, return only first three. If no, return entire list.
        if len(names) > 3:
            names = names[:3]
        return names

    #Return empty list in case of missing/malformed data
    return []

In [ ]:
df2['director'] = df2['crew_x'].apply(get_director)

features = ['cast_y', 'keywords', 'genres']
for feature in features:
    df2[feature] = df2[feature].apply(get_list)

In [ ]:
df2[['title', 'cast_y', 'director', 'keywords', 'genres']].head(3)

In [ ]:
def clean_data(x):
    if isinstance(x, list):
        return [str.lower(i.replace(" ", "")) for i in x]
    else:
        #Check if director exists. If not, return empty string
        if isinstance(x, str):
            return str.lower(x.replace(" ", ""))
        else:
            return ''

In [ ]:
features = ['cast_y', 'keywords', 'director', 'genres']

for feature in features:
    df2[feature] = df2[feature].apply(clean_data)
    # Clean the dataset into lower case and get rid of spaces.

In [ ]:
def create_soup(x):
    return ' '.join(x['keywords']) + ' ' + ' '.join(x['cast_y']) + ' ' + x['director'] + ' ' + ' '.join(x['genres'])
df2['soup'] = df2.apply(create_soup, axis=1)
# We are making metadata "soup" which is a string with all the data we want to feed our vectorizer.

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english')
count_matrix = count.fit_transform(df2['soup'])
# Count vectorizer returns how many times each word appears in a text in the form of a matrix.

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

cosine_sim2 = cosine_similarity(count_matrix, count_matrix)
# We are now performing Cosine Similarity which checks how similar two vectors are by measuring the Cosine angle between them.
# Typically used in text document similarity analysis.

In [ ]:
df2 = df2.reset_index()
indices = pd.Series(df2.index, index=df2['title'])

In [ ]:
get_recommendations('The Dark Knight Rises', cosine_sim2)

In [ ]:
get_recommendations('The Godfather', cosine_sim2)
# Congratulations again! You have another recommender system based on Keywords, Genres, and Credits.

In [ ]:
!pip install surprise

In [ ]:
from surprise import Reader, Dataset, SVD
from surprise.model_selection import cross_validate
reader = Reader()
ratings = pd.read_csv('/content/ratings_small.csv')
ratings.head()
# Reader() reads ratings

In [ ]:
reader = Reader(rating_scale=(1, 5))

In [ ]:
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)


In [ ]:
svd = SVD()
cross_validate(svd, data, measures=['RMSE', 'MAE'], cv=5, verbose=True)
# A bit to unpack here: SVD() is Single Value Decompisition. We use this to decrease the dimensionality of the matrix by taking the latent factors.
# RMSE is the Root Mean Square Error, a performance indicator for regression models. The lower, the better
# It works by measuring the average difference between values predicted by a model and the actual values. (measures accuracy.)
# MAE = Mean Absolute Error. It calculates the average magnitude of the absolute errors between the predicted and actual values.
# cv is Cross Validaton Iterator. It determines how the data will be split. For further information refer to sklearn's documentation: KFold — scikit-learn 1.6.dev0 documentation
# verbose=True just tells it to print the accuracy measure for each split.

In [ ]:
trainset = data.build_full_trainset()
svd.fit(trainset)
# We build our training set from which we can query predictions.
# .fit() actually did the training

In [ ]:
ratings[ratings['userId'] == 1]

In [ ]:
svd.predict(1, 302, 3)

In [ ]:
# For one final time, congratulations!!! I hope this helped you out and was not too difficult of a read. I tried to make this tutorial easy for beginners to understand, but I think any confusion can be solved with reading the documentation.
# Thank you for reading!

In [4]:
import sqlite3
conn = sqlite3.connect('movierex.db')

In [5]:
cursor = conn.cursor()

cursor.execute('''
CREATE TABLE IF NOT EXISTS users (
    user_id INTEGER PRIMARY KEY AUTOINCREMENT,
    name TEXT NOT NULL,
    email TEXT NOT NULL UNIQUE,
    preferences TEXT,
    top5_movies TEXT
)
''')

conn.commit()

In [ ]:
def add_user(name, email, preferences, top5_movies):
    try:
        cursor.execute('''
        INSERT INTO users (name, email, preferences, top5_movies)
        VALUES (?, ?, ?, ?)
        ''', (name, email, preferences, top5_movies))
        conn.commit()
        print(f"User {name} added successfully.")
    except sqlite3.IntegrityError as e:
        print(f"Error: {e}")


In [ ]:
def generate_recommendations(user_id):
    cursor.execute('SELECT top5_movies FROM users WHERE user_id = ?', (user_id,))
    result = cursor.fetchone()

    if not result:
        print("User not found.")
        return []

    top5_movies = result[0].split(', ')

    recommendations = set()
    for movie in top5_movies:
        recommendations.update(get_movie_recommendations(movie, 5))

    recommendations.difference_update(top5_movies)

    return list(recommendations)[:5]

In [ ]:
if __name__ == "__main__":
    name = input("Enter your name: ")
    email = input("Enter your email: ")
    preferences = input("Enter your movie preferences (comma-separated genres): ")

    print("Enter your top 5 favorite movies (one per line):")
    top5_movies = [input(f"Movie {i+1}: ") for i in range(5)]
    top5_movies_str = ", ".join(top5_movies)

    add_user(name, email, preferences, top5_movies_str)

    user_id = cursor.lastrowid  # Get the last inserted user ID
    recs = generate_recommendations(user_id)

    print(f"\nTop 5 movie recommendations for you: {recs}")

conn.close()